## 형태소 분석기 공통 메소드
- `morphs(string)` : 형태소 단위로 토큰화(tokenize)
- `nouns(string)` : 명사만 추출하여 토큰화(tokenize)    
- `pos(string)`: 품사 부착
    - 형태소 분석기 마다 사용하는 품사태그가 다르다.
        - https://konlpy-ko.readthedocs.io/ko/v0.5.2/morph/
- `tagset`: 형태소 분석기가 사용하는 품사태그 설명하는 속성.

In [ ]:
# 헌법 corpus를 분석

from nltk import Text, FreqDist
from konlpy.tag import Okt
from konlpy.corpus import kolaw
from wordcloud import WordCloud

In [ ]:
from konlpy.corpus import kolaw, kobill
kolaw.fileids(), kobill.fileids()

In [ ]:
with kolaw.open("constitution.txt") as f:
    txt = f.read()

In [ ]:
# 형태소 분석기 객체 생성. 종류별로 클래스가 제공.
from konlpy.tag import Okt, Kkma

okt = Okt()  # 형태소 분석기 객체 생성 kkma = Kkma()
#형태소 단위 토큰화
tokens = okt.morphs(txt)

print("토큰수:", len(tokens))

In [ ]:
tokens2 = okt.morphs(txt, stem=True)  # stem=True: 원형복원 - Okt의 기능
nouns= okt.nouns(txt)  #명사 토큰만 추출

In [ ]:
#품사 부착
pos_tag = okt.pos(txt)
pos_tag[:10]

In [ ]:
# OKT 기능- 비속어 처리.
# sample_txt = "반갑습니당."
sample_txt = "이것도 되나욬ㅋㅋㅋㅋㅋ"
okt.morphs(sample_txt, norm=True) #norm(normalize)= True -> 비속어 처리

# 비속어를 처리한 문장을 반환
okt.normalize(sample_txt)

In [ ]:
# 헌번 corpus를 분석

from nltk import Text, FreqDist
from konlpy.tag import Okt
from konlpy.corpus import kolaw
from wordcloud import WordCloud

with kolaw.open("constitution.txt") as fr:
    txt = fr.read()

txt[:50]

In [ ]:
# 형태소 분석기 생성
okt = Okt()
# 2. 전처리 + 토큰화
## 전처리: 명사, 동사만 추출
pos_tag = okt.pos(txt)
pos_tag[:5]

tokens =[token for token, pos in pos_tag if pos in ["Noun", "Verb"] and len(token)>1] # 명사, 동사 추출
print(len(tokens), tokens[:5])

## 왜 Subword Tokenizer가 필요한가?
- OOV(Out-Of-Vocabulary) 문제 해결
<br>: 일반 단어 단위 토크나이저는 어휘에 없는 단어가 나타나면 처리할 수 없지만, subword 단위는 이를 잘게 나누어 처리 가능.

- 어휘 크기 조절 가능
<br>: character-level보다 짧고 word-level보다 긴 중간 단위를 사용해 성능과 자원 사이의 균형을 맞춤.

- 형태소 정보 보존
<br>: 단어를 의미 있는 단위로 쪼개기 때문에 형태소나 접두사/접미사 정보를 유지할 수 있음.



In [ ]:
## <Special Token>
# 1. [UNK] - OOV(out of vocabulary)
# 2. [PAD] - 문장의 토큰 수를 맞추기 위한 padding(채우는 토큰)
# ex. 입력 데이터 - 5개로 통일  [1,2,3,4,5] , [1,2,3, [PAD], [PAD]] , [1,2,3,4,5(,6,7)-> 버림]
# 3. [CLS] - 문장의 시작을 표시 + 전체 문장의 의미를 저장하는 토큰을 사용(BERT)
# 4. [SOS] - 문장의 시작(Start of Sentence)
# 5. [EOS] - 문장의 끝(End of Sentence)
# 6. [SEP] - 문서가 여러 문장으로 구성된 경우, 문장 구분할 때.
# 7. [MASK] - 일부 토큰을 가리는 토큰   
# ex. 어휘사전에 [MASK]할 [MASK]지정 -> LLM모델 중에 MASK로 가려놓고 맞추는 방식으로 학습하는 모델이 있음

## BPE vs WordPiece vs Unigram
| 이름                           | 설명                                           | 예시 (`unhappiness`) 토큰화 |
| ---------------------------- | -------------------------------------------- | ---------------------- |
| **BPE** (Byte Pair Encoding) | 가장 빈도 높은 문자쌍부터 병합해가며 어휘 구성. 규칙 기반 병합.        | `un`, `happiness`      |
| **WordPiece**                | BPE와 유사하나 확률 기반으로 병합 결정. 주로 BERT에서 사용.       | `un`, `##happiness`    |
| **Unigram**                  | 전체 단어 집합에서 loss 최소화하는 서브워드 조합 선택. 확률적이고 유연함. | `un`, `happi`, `ness`  |


| 항목             | **BPE (Byte Pair Encoding)**                                           | **WordPiece**                                                    | **Unigram Language Model**                                          |
|------------------|------------------------------------------------------------------------|-------------------------------------------------------------------|----------------------------------------------------------------------|
| **시작 어휘**    | 문자(character) 단위                                                  | 문자(character) 단위                                              | 전체 단어 포함한 큰 어휘                                           |
| **어휘 생성 방식** | 가장 자주 등장하는 문자쌍을 반복적으로 병합                             | 조건부 확률(빈도 기반 스코어)에 따라 가장 좋은 쌍 병합            | 전체 서브워드 집합에서 loss 최소화를 기준으로 토큰 제거            |
| **토큰화 방식**   | greedy 병합 방식 (가능한 긴 토큰 우선 선택)                             | greedy 병합 방식                                                  | 확률 기반으로 가능한 여러 조합 중 가장 좋은 조합 선택              ||
| **장점**          | 빠르고 단순, 구현 쉬움                                                | BERT 등에서 효과적, BPE보다 성능 개선                             | 확률적이고 유연함, 희귀 단어에 강함, 샘플링 가능                   |
| **단점**          | 의미 없는 병합 가능성, 확률 정보 없음                                  | BPE보다 계산 복잡도 약간 높음                                     | 느린 속도, 구현 복잡도 높음                                        |
| **대표 사용 모델**| GPT, RoBERTa 등                                                      | BERT, DistilBERT                                                  | T5, ALBERT, XLNet, mT5 등                                          |
| **예시 (`unhappiness`)** | `un`, `happiness`                                               | `un`, `##happiness`                                               | `un`, `happi`, `ness`                                               |


## 학습방식
| 방식            | 학습 기준 요약                    | 확률 사용 여부  | 병합 or 제거   |
| ------------- | --------------------------- | --------- | ---------- |
| **BPE**       | 가장 자주 등장하는 문자(또는 토큰) 쌍을 병합  | ❌ (빈도 기반) | 병합         |
| **WordPiece** | 조건부 확률(점수)이 높은 서브워드 쌍을 병합   | ✅         | 병합         |
| **Unigram**   | 전체 문장 확률을 최대화하는 서브워드 조합을 선택 | ✅         | 제거 (prune) |


| 모델       | BPE                                                                                          | WordPiece                                                                                  | Unigram                                                                                                       |
|------------|-----------------------------------------------------------------------------------------------|---------------------------------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------|
| Training   | 작은 vocabulary부터 시작하고, 토큰을 병합하는 규칙을 학습. 모든 문자를 서브워드로 처리하고, 가장 빈도가 높은 바이트 페어를 병합하면서 새로운 서브워드 집합을 생성 | 작은 vocabulary부터 시작하고 토큰을 병합하는 규칙을 학습. 단어 목록을 가지고 시작하며, 주어진 텍스트에 있는 모든 단어를 자주 나오는 부분어로 분할하고, 이들을 병합하여 새로운 서브워드를 생성 | large vocabulary부터 시작하고 토큰을 삭제하는 규칙을 학습. 모든 단어를 하나의 서브워드로 처리하고, 빈도에 따라 새로운 서브워드 생성. 빈도에 따라 샘플링되는데, 효율적인 학습을 위해 랜덤성을 추가 |
| Training step | 가장 일반적인 쌍에 해당하는 토큰을 병합                                                   | 가장 빈번한 쌍에 대한 스코어를 기반으로 토큰을 병합                                         | 전체 corpus에서 loss를 최소화하는 단어의 모든 토큰을 제거                                                    |
| Learns     | 병합 규칙, vocab                                                                             | vocab                                                                                       | 각 토큰의 점수와 vocab                                                                                       |
| Encoding   | 단어를 문자로 분할하고 학습 중에 학습된 병합을 적용                                            | 어휘의 처음부터 시작하여 가장 긴 하위 단어를 찾은 다음 나머지 단어에 대해서도 동일한 작업 수행 | 훈련 중에 학습된 점수를 사용하여 토큰으로 분할할 가능성이 가장 높은 항목을 찾음                               |


In [ ]:
import time
# Tokenizer
from tokenizers import Tokenizer
from tokenizers.models import BPE
# subword 알고리즘을 적용하기 전에 어떻게 나눠놓을 것인지
from tokenizers.pre_tokenizers import Whitespace
# Trainer (학습)
from tokenizers.trainers import BpeTrainer

# 토크나이저 생성
# subword 알고리즘을 구현한 Tokenizer의 객체를 넣어 생성.
# unk_token = OOV(Out of Vocabulary  = UNknown) 단어(토큰)을 처리할 토큰을 지정.
tokenizer = Tokenizer(BPE(unk_token='[UNK]'))
# pre tokenizer를 등록
tokenizer.pre_tokenizer = Whitespace()  # 공백을 기준으로 미리 토큰화해 놓는다. 
#tokenizer를 학습하는 Trainer 객체 -> Tokenizer 알고리즘 별로 Trainer 클래스가 제공됨.
# initializer에 어떻게 학습시킬지 설정정
trainer = BpeTrainer(
    vocab_size = 10000, #어휘사전의 최대 크기(사전에 넣을 고유 토큰의 최대 개수.)
    min_frequency = 10, # 사전에 넣을 토큰의 **최소 출현 횟수** (빈도 수)
    special_tokens = ["[UNK]", "[PAD]"], # 어휘 사전에 추가할 특수(목적)토큰(Special token)들 지정.
    # 이 중 unk_token은 반드시 설정해야함.
)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import WordPieceTrainer

# 토크나이저 생성
tokenizer2 =  Tokenizer(WordPiece(unk_token="[UNK]"))
# Pre tokenizer
tokenizer2.pre_tokenizer = Whitespace()
# Trainer 생성
trainer = WordPieceTrainer(
    vocab_size=20000,
    special_tokens=["[UNK]", "[PAD]", "[SEP]", "[SOS]", "[EOS]"],
    min_frequency=10
)

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import Unigram
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.trainers import UnigramTrainer

# 토크나이저 생성
tokenizer3 =  Tokenizer(Unigram())
# Pre tokenizer
tokenizer3.pre_tokenizer = Whitespace()
# Trainer 생성
trainer = UnigramTrainer(
    vocab_size=20000,
    special_tokens=["[UNK]", "[PAD]", "[SEP]", "[SOS]", "[EOS]"],
    min_frequency=10
)